In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
import os

from taogpt.orchestrator import *
from taogpt.utils import *
from taogpt.llm_model import LangChainLLM
from taogpt.prompts import PromptDb
import taogpt.utils as utils

utils.enable_debugging(0)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
TEMPERATURE = 0.7

In [4]:
with open(os.path.join(os.environ['HOME'], '.ssh', 'openai-zillow.json'), 'r') as f:
    credentials = json.load(f)
os.environ["OPENAI_API_KEY"] = credentials['key']
os.environ["OPENAI_API_BASE"] = credentials['url']
llm3_5 = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=TEMPERATURE)
llm4 = ChatOpenAI(model_name='gpt-4-32k', temperature=TEMPERATURE)

conversation = ConversationChain(llm=llm3_5)
conversation.predict(input="What's your model version?")

"I am using OpenAI's GPT-3 model."

In [5]:
llm3_5.model_name, llm4.model_name

('gpt-3.5-turbo-16k', 'gpt-4-32k')

In [7]:
prompts = PromptDb.load_defaults()
logger = MarkdownLogger('logs/taogpt_log.md')
config = Config(
    ask_user_before_execute_codes=False,
    ask_user_questions_in_one_prompt=True,
    pause_after_initial_solving_expansion=True,
    first_expansion=1,
    initial_expansion=4,
    max_tree_branches=6,
    max_tokens=5000,
    check_final=True
)
executor = Orchestrator(
    config=config,
    llm=LangChainLLM(llm4, logger=logger),
    prompts=prompts,
    markdown_logger=logger,
    # sage_llm=LangChainLLM(llm4, logger=logger),
)

experiment_name = 'example'
executor.start("""
Solve this 4x4 Sudoku.

```text
_ 3 _ 1
1 _ _ 3
2 _ 3 _
3 4 _ 2
```

(`_` represents an empty cell.)

Solve in your head. Do NOT use Python programming.
""", analyze_first=True)

Pause: Pause after initial solving expansion

In [32]:
executor._chain[11].step.description

'The fourth row of the Sudoku puzzle is `3 4 _ 2`. The missing numbers in this row are 1 and 4. \n\nSince the number 1 is already present in the second column (in the third row), the third cell of the fourth row cannot be 1. Therefore, it must be 4. \n\nSo, the fourth row after filling in the missing numbers is `3 4 1 2`.'

In [33]:
executor._chain = executor._chain[:11]

In [34]:
executor.chain[-1].step.description

'Tao, some critic says your last step answer has errors while others think it\'s OK. Please check the steps done so far.\nIf there are indeed errors, respond using the `BACKTRACK_ON_ERROR` strategy, otherwise proceed to step \n\n* None. This is the final step.\n\nfollowing **strictly** the "Problem Solving Instructions" and templates above. Be sure to start with one of the \nheadings: `# I_WILL_ANSWER_DIRECTLY`, `# LET_ME_ASK_THE_PYTHON_GENIE`, `# BACKTRACK_ON_ERROR`, \n`# I_NEED_TO_ASK_SOME_QUESTIONS_BEFORE_I_PROCEED`, or `# HERE_IS_MY_STEP_BY_STEP_PLAN`.\n\n'

In [18]:
# executor._prompts = PromptDb.load_defaults()

In [35]:
executor.resume(10000, additional_tokens_for_smarter_llm=0, unblock_initial_expansion=True)

extend token allowance by 10000 to 55000
extend token allowance for smarter LLM by 0 to 5000


In [14]:
logger = MarkdownLogger(f'examples/{experiment_name}.final.md')
logger.log_conversation(executor.show_conversation_thread(with_header=True))
logger.log(f"**total tokens**: {executor.llm.total_tokens}")
